In [1]:
import requests
import json
from pprint import PrettyPrinter
import tensorflow as tf

## Make prediction request

In [18]:
# URL untuk mengirim request ke model yang berjalan di Docker
url = "https://proyek-kedua-ydrkli3j2q-uc.a.run.app/v1/models/spam-detection-model:predict"

# Contoh pesan yang akan diuji
test_message = "Congratulations! You've won a $1000 Walmart gift card. Go to http://bit.ly/123456 to claim now."

# Membuat TFRecord serialized example
def create_tfrecord_example(message):
    feature = {
        "Message": tf.train.Feature(bytes_list=tf.train.BytesList(value=[message.encode()]))
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

# Membuat payload untuk request
payload = {
    "signature_name": "serving_default",
    "instances": [{"examples": create_tfrecord_example(test_message).decode('ISO-8859-1')}]
}

# Mengirim POST request ke server model
response = requests.post(url, data=json.dumps(payload))

# Mengecek dan menampilkan hasil prediksi
if response.status_code == 200:
    prediction = response.json()['predictions'][0][0]
    label = "Spam" if prediction >= 0.5 else "Not Spam"
    print(f"Prediction: {prediction} ({label})")
else:
    print(f"Error: {response.status_code}, {response.text}")


Prediction: 0.999254704 (Spam)
